In [1]:
import json;
import pandas as pd;
import scipy;
from scipy import stats;
# from apyori import apriori;
import matplotlib.pyplot as plt;
import numpy as np;
import sklearn
from sklearn.datasets import load_iris
from sklearn import tree

data_path = 'okcupid_11_25_22.csv'
df = pd.read_csv(data_path)

# drop columns you're not using, convert sex to bool
df.loc[ df['sex'] == 'f', 'sex'] = 0
df.loc[ df['sex'] == 'm', 'sex'] = 1
df = df.filter(items = ['age', 'body_type', 'drinks', 'height', 'smokes', 'education', 'orientation', 'sign', 'sex'])
df

,age,body_type,drinks,height,smokes,education,orientation,sign,sex
0,22,a little extra,socially,75.0,sometimes,working on college/university,straight,gemini,1
1,35,average,often,70.0,no,working on space camp,straight,cancer,1
2,38,thin,socially,68.0,no,graduated from masters program,straight,pisces but it doesn&rsquo;t matter,1
3,23,thin,socially,71.0,no,working on college/university,straight,pisces,1
4,29,athletic,socially,66.0,no,graduated from college/university,straight,aquarius,1
...,...,...,...,...,...,...,...,...,...
59937,59,NaN,socially,62.0,no,graduated from college/university,straight,cancer and it&rsquo;s fun to think about,0
59938,24,fit,often,72.0,no,working on college/university,straight,leo but it doesn&rsquo;t matter,1
59939,42,average,not at all,71.0,no,graduated from masters program,straight,sagittarius but it doesn&rsquo;t matter,1
59940,27,athletic,socially,73.0,trying to quit,working on college/university,straight,leo and it&rsquo;s fun to think about,1


In [2]:
# drop rows with too many blank values
df = df[df.isnull().sum(axis=1) < 3]

# fill blank values with prefer not to say
df.fillna('rather not say', inplace=True)
            
attraction_dict = {'attractive': ['athletic', 'curvy', 'fit','full figured', 'jacked'],
                   'neutral': ['a little extra', 'average', 'thin',],
                   'unattractive': ['overweight', 'skinny', 'used up']}

education_dict = {'some_college': ['dropped out of college/university', 'dropped out of two-year college', 'graduated from two-year college', 
                                   'two-year college', 'working on college/university', 'working on two-year college'],
                  'some_high_school': ['dropped out of high school', 'working on high school'], 
                  'post_secondary': ['graduated from law school', 'graduated from masters program', 'graduated from med school',
                                       'graduated from ph.d program', 'law school', 'masters program', 'med school', 'ph.d program'],
                  'college': ['college/university', 'dropped out of masters program', 'dropped out of med school', 'dropped out of ph.d program',
                              'graduated from college/university', 'dropped out of law school', 'working on law school', 'working on masters program',
                              'working on med school', 'working on ph.d program'],
                  'high_school': ['graduated from high school', 'high school'],
                  'space_camp': ['graduated from space camp', 'dropped out of space camp', 'space camp', 'working on space camp']}

frequency_dict = {'a_lot': ['desperately', 'often', 'very often', 'trying to quit', 'yes'],
                  'never': ['not at all', 'no' ],
                  'sometimes': ['rarely', 'socially', 'sometimes', 'when drinking']}

answer_bank = ['athletic', 'curvy', 'fit','full figured', 'jacked', 'a little extra', 'average', 'thin', 'overweight', 'skinny', 'used up', 
               'dropped out of college/university', 'dropped out of two-year college', 'graduated from two-year college', 
               'two-year college', 'working on college/university', 'dropped out of law school', 'working on two-year college', 'dropped out of high school', 'working on high school', 
               'dropped out of high school', 'working on high school', 'graduated from law school', 'graduated from masters program', 'graduated from med school',
               'graduated from ph.d program', 'law school', 'masters program', 'med school', 'ph.d program','college/university', 'dropped out of masters program', 'dropped out of med school', 'dropped out of ph.d program',
               'graduated from college/university', 'graduated from high school', 'high school', 'desperately', 'often', 'very often', 'trying to quit', 'yes', 'not at all', 'no', 'rarely', 'socially', 'sometimes', 'when drinking', 'working on ph.d program', 'working on law school', 'working on masters program', 'working on med school']

C:\Users\13369\AppData\Local\Temp\ipykernel_18448\57513329.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna('rather not say', inplace=True)


In [3]:
dict_list = [frequency_dict, education_dict, attraction_dict]
columns_list = ['body_type', 'drinks', 'smokes', 'education']

for c in columns_list:
     for entry in df[c]:
        if 'space camp' in entry:
            df.loc[ df[c] == entry, c] = 'space camp'
        else:
             for d in dict_list:
                for key in d:
                    if entry in d[key]:
                        if entry in answer_bank:
                            df.loc[ df[c] == entry, c] = key
                            answer_bank.remove(entry)
            # entry = presentInMat(entry, dict_list)
    

help_dict = {}
help_df = df['sign']
sign_dict = {'aries': 1, 'taurus': 2, 'gemini': 3, 'cancer': 4, 'leo': 5, 'virgo': 6, 'libra': 7, 'scorpio': 8, 'sagittarius': 9, 'capricorn': 10, 'aquarius': 11, 'pisces': 12 }
for i in range(df.shape[0]):
    sign_entry = help_df.iloc[i] # take an entry's sign
    if sign_entry not in help_dict: # if the entry's sign is not in the help_dictionary  
        for sign in sign_dict: # for every sign in the sign_dict
            if sign in sign_entry:  #if the entry's sign contains one of the official sign
                help_dict[sign_entry] = sign_dict[sign] #the help dict has a sign entry with the official sign in it

# all the entries with the different types of sign entries are now converted to offical sign numbers
for key in help_dict: 
    df.loc[ df['sign'] == key, 'sign'] = help_dict[key] 
    for i in range(df.shape[0]):
        if type(df['sign'].iloc[i]) == "<class 'str'>": 
            print(df['sign'].iloc[i])
df = df[df['sign'] != 'rather not say']
df = df[df['height'] != 'rather not say']

In [4]:
df = pd.get_dummies(data=df, columns=['body_type', 'drinks', 'smokes', 'education', 'orientation'])

In [5]:
df.columns

Index(['age', 'height', 'sign', 'sex', 'body_type_attractive',
       'body_type_neutral', 'body_type_rather not say',
       'body_type_unattractive', 'drinks_a_lot', 'drinks_never',
       'drinks_rather not say', 'drinks_sometimes', 'smokes_a_lot',
       'smokes_never', 'smokes_rather not say', 'smokes_sometimes',
       'education_college', 'education_high_school',
       'education_post_secondary', 'education_rather not say',
       'education_some_college', 'education_some_high_school',
       'education_space camp', 'orientation_bisexual', 'orientation_gay',
       'orientation_straight'],
      dtype='object')

In [13]:
df


,age,height,sign,sex,body_type_attractive,body_type_neutral,body_type_rather not say,body_type_unattractive,drinks_a_lot,drinks_never,...,education_college,education_high_school,education_post_secondary,education_rather not say,education_some_college,education_some_high_school,education_space camp,orientation_bisexual,orientation_gay,orientation_straight
0,22,75.0,3,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,35,70.0,4,1,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
2,38,68.0,12,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,23,71.0,12,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,29,66.0,11,1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59937,59,62.0,4,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
59938,24,72.0,5,1,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
59939,42,71.0,9,1,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
59940,27,73.0,5,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [35]:
from sklearn.model_selection import train_test_split
['age', 'body_type', 'drinks', 'height', 'smokes', 'education', 'orientation', 'sex']
X = df.filter(items = ['age', 'body_type', 'drinks', 'height', 'smokes', 'education', 'orientation', 'sex']).values
y = df['sign'].values
y=y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

decision_tree_best = tree.DecisionTreeClassifier() 
decision_tree_best.fit(X_train, y_train)
y_preds = decision_tree_best.predict(X_test)

scoresVC_p = precision_score(y_test, y_preds, average=None)
scoresVC_r = recall_score(y_test, y_preds, average=None) 

print("Decision Tree Precision: ", scoresVC_p)
print("Recall: ", scoresVC_r)

lr_best = LogisticRegression() 
lr_best.fit(X_train, y_train)
y_preds = lr_best.predict(X_test)

scoresVC_p_lr = precision_score(y_test, y_preds, average=None)
scoresVC_r_lr = recall_score(y_test, y_preds, average=None) 

print("Logistic Regression Precision: ", scoresVC_p_lr)
print("Recall: ", scoresVC_r_lr)

knn_best = KNeighborsClassifier() 
knn_best.fit(X_train, y_train)
y_preds = knn_best.predict(X_test)

scoresVC_p_knn = precision_score(y_test, y_preds, average=None)
scoresVC_r_knn = recall_score(y_test, y_preds, average=None) 

print("KNN Precision: ", scoresVC_p_knn)
print("Recall: ", scoresVC_r_knn)


Decision Tree Precision:  [0.09182531 0.08422939 0.09745127 0.07254464 0.10009099 0.07932961
 0.09346734 0.08613139 0.07927928 0.05211726 0.07598784 0.09153713]
Recall:  [0.09963548 0.11547912 0.14656144 0.07850242 0.12746234 0.09010152
 0.10979929 0.07375    0.05759162 0.02200825 0.03242542 0.06874189]


C:\Users\13369\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Logistic Regression Precision:  [0.         0.         0.0862796  0.         0.09060606 0.
 0.         0.         0.         0.         0.         0.        ]
Recall:  [0.         0.         0.29988726 0.         0.69293163 0.
 0.         0.         0.         0.         0.         0.        ]
KNN Precision:  [0.0856998  0.07830986 0.08436945 0.08       0.09672619 0.09028961
 0.09688013 0.0770751  0.07823961 0.06798867 0.10158014 0.09048724]
Recall:  [0.20534629 0.17076167 0.10710259 0.07729469 0.07531866 0.06725888
 0.06965762 0.04875    0.04188482 0.03301238 0.05836576 0.05058366]


In [ ]:
df.columns


In [37]:
print("DT F1 Scores: ", (2*scoresVC_p*scoresVC_r)/(scoresVC_r+scoresVC_p))
print("DT (Avg. Precision, Avg. Recall): ", (np.mean(scoresVC_p), np.mean(scoresVC_r)))
print("LR F1 Scores: ", (2*scoresVC_p_lr*scoresVC_r_lr)/(scoresVC_r_lr+scoresVC_p_lr))
print("LR (Avg. Precision, Avg. Recall): ", (np.mean(scoresVC_p_lr), np.mean(scoresVC_r_lr)))
print("KNN F1 Scores: ", (2*scoresVC_p_knn*scoresVC_r_knn)/(scoresVC_r_knn+scoresVC_p_knn))
print("KNN (Avg. Precision, Avg. Recall): ", (np.mean(scoresVC_p_knn), np.mean(scoresVC_r_knn)))

DT F1 Scores:  [0.0955711  0.09740933 0.11706439 0.07540603 0.11213048 0.08437314
 0.1009772  0.07946128 0.06671721 0.03094778 0.04545455 0.07851852]
DT (Avg. Precision, Avg. Recall):  (0.0836659548501529, 0.08517156669956755)
LR F1 Scores:  [       nan        nan 0.13400504        nan 0.16025727        nan
        nan        nan        nan        nan        nan        nan]
LR (Avg. Precision, Avg. Recall):  (0.014740471533368063, 0.08273490785532234)
KNN F1 Scores:  [0.12093023 0.10737737 0.09438649 0.07862408 0.08469055 0.07709091
 0.08104396 0.05972435 0.05456095 0.04444444 0.07413509 0.06489185]
KNN (Avg. Precision, Avg. Recall):  (0.08563714885837341, 0.08377808422996708)


C:\Users\13369\AppData\Local\Temp\ipykernel_18448\3917437107.py:3: RuntimeWarning: invalid value encountered in true_divide
  print("LR F1 Scores: ", (2*scoresVC_p_lr*scoresVC_r_lr)/(scoresVC_r_lr+scoresVC_p_lr))
